# Generating a random fermionic guassian statevector

In [1]:
using Pkg
Pkg.add("StatsBase")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [1]:
using LinearAlgebra

#### Generating a random Hermitian matrix (A)

In [11]:
function random_hermitian(N)
    A = randn(ComplexF64, N, N)
    H = (A + A') / 2
    return H
end
A = random_hermitian(1024)

1024×1024 Matrix{ComplexF64}:
  -1.21017+0.0im          -0.309466+0.336151im    …   -0.019518+0.158111im
 -0.309466-0.336151im     -0.417458+0.0im             0.0491206-0.258792im
 -0.445915-0.583953im      0.301703-0.210804im         0.356704-0.0568846im
 -0.660574-0.245032im     0.0530527-0.0145032im       -0.218359+1.15003im
   1.30166-0.46262im      -0.396141+0.273695im       -0.0346118-0.402597im
  0.685719+1.05222im      -0.357303-0.435371im    …   -0.676776+0.166726im
 -0.960061-0.645463im     -0.530736+0.15431im          0.184432-0.121276im
 -0.198323-0.0578778im    -0.130443+0.209938im        -0.155044-0.573661im
 -0.269838+0.458594im      0.845215-0.553036im        -0.432985-0.448655im
 -0.485633+0.220702im      0.518129-0.0035811im       -0.483053-0.311638im
          ⋮                                       ⋱  
 0.0756083-0.0685084im    0.0385101+0.186192im    …   0.0740151-0.240705im
 0.0328126-0.610523im   -0.00253947-0.00599347im       0.273999-1.44749im
 -0.103577+0.2887

#### Generating a random skew symmetric matrix (B)

In [12]:
function random_skew_symmetric_complex(N)
    A = randn(ComplexF64, N, N)
    return (A - transpose(A)) / 2
end
B  = random_skew_symmetric_complex(1024)

1024×1024 Matrix{ComplexF64}:
        0.0+0.0im          0.300273+0.301532im   …   -0.469705+0.894988im
  -0.300273-0.301532im          0.0+0.0im             0.374988-0.154563im
   0.534967+0.640312im   -0.0194911-0.420146im        -0.33352-0.222052im
  -0.480616-0.442134im     0.273249-1.05511im        -0.896276+0.630676im
  -0.015876-0.426539im     0.197663+0.520604im        0.410767+0.138671im
    1.36372-0.716379im    -0.353179+0.188127im   …   0.0337865+0.650967im
  -0.268867-0.104553im     0.433657+0.0392657im       0.278236+0.0450626im
   0.452058+0.372497im   -0.0902551-1.09831im         0.747264+0.0764823im
   -1.13674+0.280056im    -0.471873+0.325018im      -0.0320742-0.242353im
  -0.163249-0.0670833im   -0.277138-0.0136684im       0.828956+0.288822im
           ⋮                                     ⋱  
  -0.764786+0.0970625im   0.0611746+0.0733633im  …    0.210953+0.0766554im
   0.114526-0.156259im    0.0779839+0.0277641im       0.442453-0.682931im
  -0.438889+0.826207im    

#### Constructing Hamiltonian based on A and B

In [13]:
function construct_H(A::AbstractMatrix, B::AbstractMatrix)
    n, m = size(A)
    @assert size(B) == (n, m) "A and B must be the same size"

    H = Matrix{eltype(A)}(undef, 2n, 2m)

    # Fill in blocks directly
    H[1:n, 1:m]     .= -conj.(A)
    H[1:n, m+1:2m]  .= B
    H[n+1:2n, 1:m]  .= -conj.(B)
    H[n+1:2n, m+1:2m] .= A

    return H
end
H = construct_H(A, B)

2048×2048 Matrix{ComplexF64}:
   1.21017+0.0im          0.309466+0.336151im   …   -0.469705+0.894988im
  0.309466-0.336151im     0.417458+0.0im             0.374988-0.154563im
  0.445915-0.583953im    -0.301703-0.210804im        -0.33352-0.222052im
  0.660574-0.245032im   -0.0530527-0.0145032im      -0.896276+0.630676im
  -1.30166-0.46262im      0.396141+0.273695im        0.410767+0.138671im
 -0.685719+1.05222im      0.357303-0.435371im   …   0.0337865+0.650967im
  0.960061-0.645463im     0.530736+0.15431im         0.278236+0.0450626im
  0.198323-0.0578778im    0.130443+0.209938im        0.747264+0.0764823im
  0.269838+0.458594im    -0.845215-0.553036im      -0.0320742-0.242353im
  0.485633+0.220702im    -0.518129-0.0035811im       0.828956+0.288822im
          ⋮                                     ⋱  
  0.764786+0.0970625im  -0.0611746+0.0733633im      0.0740151-0.240705im
 -0.114526-0.156259im   -0.0779839+0.0277641im  …    0.273999-1.44749im
  0.438889+0.826207im   -0.0100062+1.2921

#### Checking if the generated matrix is Hermitian

In [14]:
ishermitian(H)

true

##### Constructing rho

In [15]:
function build_rho(H::Matrix{ComplexF64})
    exp_H = exp(-H)           
    Z = tr(exp_H)                
    rho = exp_H / Z             
    return rho
end
rho = build_rho(H)


2048×2048 Matrix{ComplexF64}:
  0.000238328+0.0im          …    -6.2024e-6-0.000221598im
   5.00221e-5-0.00014411im       -8.62595e-5-3.86008e-5im
  -4.36494e-5-4.995e-5im          5.70648e-5+0.000332016im
   1.75094e-5+0.00017093im        0.00047329-0.000183868im
   9.90781e-6-0.00010147im      -0.000170651+2.86602e-5im
   4.57028e-5-0.00018719im   …  -0.000225226-0.000157629im
 -0.000126374-9.14804e-5im       -9.22026e-5+0.000111875im
  -5.24248e-5+9.47367e-5im       0.000134278+0.000225361im
  0.000172677+4.26422e-5im        9.10329e-5-0.000484498im
    6.6121e-5-4.93722e-5im      -0.000246242+0.000142495im
             ⋮               ⋱  
  -2.76319e-5+0.000132661im      -3.59745e-5-6.49509e-6im
   5.66139e-6+5.06112e-6im   …    8.29844e-5-0.000184052im
  0.000129886-9.37108e-5im       -0.00013448-0.000284161im
  0.000165793+5.94278e-5im      -0.000139871-0.000281329im
  -8.26815e-5+1.0517e-5im         7.23226e-5+0.000132971im
 -0.000136026-0.000126813im      -3.95192e-5+0.00025269

#### Making rho a pure state in a larger Hilbert space

In [7]:
function purification_rho(rho::Matrix{ComplexF64})
    vals, vecs = eigen(rho)  # rho = vecs * Diagonal(vals) * vecs'

    d = size(rho, 1)
    psi = zeros(ComplexF64, d^2)  # Purified state vector lives in d x d space

    for k in 1:d
        pk = vals[k]
        if pk ≈ 0
            continue  # Skip zero-eigenvalue components
        end
        psi_k = vecs[:, k]
        tr_k = zeros(ComplexF64, d)
        tr_k[k] = 1.0  # |k⟩ in ancilla space


        psi += sqrt(pk) * kron(psi_k, tr_k)
    end

    # Normalize Ψ (should already be normalized, but numerically might not be)
    psi ./= norm(psi)

    return psi
end
statevector = purification_rho(rho)

DomainError: DomainError with -2.108425289312159e-16:
sqrt was called with a negative real argument but will only return a complex result if called with a complex argument. Try sqrt(Complex(x)).

In [35]:
open("FGS_Gen.txt", "w") do file
    for i in 1:length(statevector)
        real_part = real(statevector[i])
        imag_part = imag(statevector[i])
        write(file, "$real_part + $imag_part im\n")
    end
end

#### Sample from the density matrix

In [16]:
using LinearAlgebra
using StatsBase
function sample_from_rho(rho::Matrix{ComplexF64})
    vals, vecs = eigen(Hermitian(rho))  # ρ = vecs * Diagonal(vals) * vecs'

    probs = abs.(vals) ./ sum(abs.(vals))

    # Sample index k with probability λ_k
    k = sample(1:length(probs), Weights(probs))

    ψ = vecs[:, k]
    ψ ./= norm(ψ)
    return ψ
end


sample_from_rho (generic function with 1 method)

In [17]:
SVEC = sample_from_rho(rho)

2048-element Vector{ComplexF64}:
  -0.014569251574443514 - 0.002076344160242297im
 -0.0018992175909872471 - 0.01703037530363529im
  0.0005993610391282785 + 0.009740379731596195im
  -0.009978951361031276 + 0.008810167206586903im
   0.006301517836755445 - 0.009026630498592295im
   0.025113331135152107 - 0.005810952242168229im
   -0.01209335436798149 - 0.011906690439904829im
   0.009972828375548318 - 0.015683697910818768im
   0.013230901637541431 - 0.012300460833212063im
   0.014312363922931887 - 0.013447234774494367im
                        ⋮
  -0.018805295624435506 + 0.00038329616959689776im
   0.030272356694615452 - 0.029944682850995153im
   0.008055749664027912 - 0.030215596599089952im
   -0.01314175844554029 - 0.009233457498672137im
  0.0012455312396189649 + 0.0023587466490558647im
    0.00940608263888436 + 0.01989817903830282im
   0.005486481497796116 - 4.6789847016233867e-5im
   0.009758291338277136 + 0.00029404099150054944im
  -0.010428620279189704 - 0.0im

In [18]:
open("FGS_Gen_2.txt", "w") do file
    for i in 1:length(SVEC)
        real_part = real(SVEC[i])
        imag_part = imag(SVEC[i])
        write(file, "$real_part + $imag_part im\n")
    end
end